## PART 2 : STORING ANCHORS

In [3]:
import os
import h5py 
import numpy as np
from tqdm import tqdm

anchors = [(12,12,2,2),(19,20,3,4),(22,22,5,5),(34,35,6,7),(42,42,8,8),(49,50,9,10)]

feature_folder = 'FEATURE_MAPS'
anchor_folder = 'ANCHORS'

for fm in tqdm(os.listdir(feature_folder)):
    feature_map = h5py.File(feature_folder+"/"+fm,'r+')
    feature_map = np.array(feature_map.get('default'))
    [ n , n_i, n_j, n_k ] = feature_map.shape
    
    cnt = 1    
    
    for w_l, w_r, h_l, h_r in anchors:
        for i in range( h_l, n_i-h_r, 10):
            for j in range( w_l, n_j-w_r, 10):
                top_left_x = i - h_l
                top_left_y = j - w_l
                bottom_right_x = i + h_r
                bottom_right_y = j + w_r
                
                output_name = anchor_folder+'/'+fm.split('.')[0]+'_anchor_'+str(cnt)+'.h5'
                if os.path.isfile(output_name):
                    continue
                with h5py.File(output_name,'w') as hf:
                    anchor = feature_map[ :, top_left_x : bottom_right_x, top_left_y : bottom_right_y, :]                
                    hf.create_dataset('anchor_'+str(cnt), data=anchor, compression="gzip")
                    cnt += 1
                

100%|██████████| 126/126 [1:04:47<00:00, 30.53s/it]
